MPCA checkups

In [3]:
DECLARE @PeriodStart as varchar(100) = '20241111 00:00:00.000'
DECLARE @PeriodEndPlusOne as varchar(100) = '20241118 00:00:00.000'

PRINT 'Missing SPN'
    SELECT [Registration group ID] as 'Group with Missing SPN'
    FROM [progresSSISdb].[dbo].[RG]
    where year([Created on]) > 2023
    and [Group size] > 0
    and [Registration group ID] not in (
      'FC3-24-3438751','FC3-24-3340634','FC3-24-3579835','FC3-24-3659386','FC3-24-3385600', 'FC3-24-3946749', 'FC3-24-3196559', 'FC3-24-4678593',
      'FC3-24-3284595', 'FC3-24-4678544', 'FC3-24-4595411', 'FC3-24-4678507', 'FC3-24-3709350', 'FC3-24-3199124', 'FC3-24-3752242', 'FC3-24-3558059', 'FC3-24-3703923'
      )
      -- groups to be moved to a table for easier future reference
EXCEPT
    SELECT [Registration group ID]
    FROM [progresSSISdb].[dbo].[SPN]

PRINT 'IND duplicates by Tax ID'
SELECT [Legacy ID] as 'Duplicate Tax ID'
        FROM [progresSSISdb].[dbo].[Ind]
        where [Legal Status] <> 'not of concern'
        and [Legacy ID] not in ('na','NTID A','NTID R','NTID D','NTID L','NTID LB')
        and [Legacy ID] <> ''
        and [Created On] < @PeriodStart
INTERSECT
    SELECT [Legacy ID]
        FROM [progresSSISdb].[dbo].[Ind]
        where [Legal Status] <> 'not of concern'
        and [Legacy ID] not in ('na','NTID A','NTID R','NTID D','NTID L','NTID LB') 
        and [Legacy ID] <> ''
        and [Created On] >= @PeriodStart
        and [Created On] < @PeriodEndPlusOne

PRINT 'IND duplicates by bio data'
SELECT [Individual ID]
      ,[Full Name] as 'Duplicate Name'
      ,[Registration Group ID]
      ,[Date of Birth] 'Duplicate DoB'
  FROM [progresSSISdb].[dbo].[Ind]
  where CONCAT([Full Name],[Date of Birth]) in (
      SELECT CONCAT([Full Name],[Date of Birth]) FNDB
        FROM [progresSSISdb].[dbo].[Ind]
        where [Registration Date] < @PeriodStart
        and [Legal Status] <> 'not of concern'
      INTERSECT
      SELECT CONCAT([Full Name],[Date of Birth]) FNDB
        FROM [progresSSISdb].[dbo].[Ind]
        where [Registration Date] >= @PeriodStart
        and [Registration Date] < @PeriodEndPlusOne
        and [Legal Status] <> 'not of concern'
  )
  order by [Full Name]

PRINT 'Phone number duplicates'
SELECT [Individual ID]
      ,[Full Name]
      ,[Registration Group ID]
      ,[Legal Status]
      ,[Age]
      ,[Sex]
      ,[Legacy ID]
      ,[Relationship to Focal Point]
      ,[Phone Number] as 'Duplicate phone number'
      ,[Created On]
  FROM [progresSSISdb].[dbo].[Ind]
  where [Phone Number] in (
      SELECT [Phone Number]
        FROM [progresSSISdb].[dbo].[Ind]
        where [Registration Date] < @PeriodStart
        and [Phone number] <> '+380000000000'
    INTERSECT
      SELECT [Phone Number]
        FROM [progresSSISdb].[dbo].[Ind]
        where [Registration Date] >= @PeriodStart
        and [Registration Date] < @PeriodEndPlusOne
        and [Phone number] <> '+380000000000'
    )
  order by [Phone Number]

PRINT 'RCp duplicates'
SELECT [Reception ID]
      ,[Group Size]
      ,[Government Form Number]
      ,[Reception Location]
      ,[Registration Group ID]
  FROM [progresSSISdb].[dbo].[RCP]
  join [progresSSISdb].[dbo].[Ind] on [progresSSISdb].[dbo].[Ind].[Legacy ID] = [progresSSISdb].[dbo].[RCP].[Government Form Number]
  where [Government Form Number] in (
        SELECT [Government Form Number]
        FROM [progresSSISdb].[dbo].[RCP]
        INTERSECT
        SELECT [Legacy ID]
            FROM [progresSSISdb].[dbo].[Ind]
            where [Legal Status] <> 'not of concern'
            and [Legacy ID] not in ('na','NTID A','NTID R','NTID D','NTID L','NTID LB') 
            and [Legacy ID] <> ''
        and [Created On] >= @PeriodStart
        and [Created On] < @PeriodEndPlusOne
  )
  order by [Reception Location]

  PRINT 'IDP More than 6 months ago (excluding emergency)'
  SELECT [Individual ID]
      ,[Registration Group ID]
      ,[Legal Status]
      ,[Arrival date]
      ,cast([Created on] as date) as 'Created on'
      ,DATEDIFF(day,[Arrival date],cast([Created on] as date)) as 'Difference in days is greater than 180'
  FROM [progresSSISdb].[dbo].[Ind]
  where 
    [Legal Status] in ('IDP-like', 'Internally displaced person')
    and
    DATEDIFF(day,[Arrival date],cast([Created on] as date)) > 180
    AND
    [Registration Group ID] not in (
        SELECT [Registration group]
        FROM [progresSSISdb].[dbo].[Events]
        where Event = 'ukidp-reg13'
    )
    and [Created On] >= @PeriodStart
    and [Created On] < @PeriodEndPlusOne

PRINT 'Documents'
PRINT 'Duplicates'

SELECT [Individual]
      ,[Full name]
      ,[Date of birth]
      ,[Legal Status]
      ,[Document category]
      ,[Document type]
      ,[Document sub type]
      ,[Number] 'Number-Duplicate'
      ,[Date of issue]
      ,[Date of expiry]
      ,[Is original]
      ,[Is official]
      ,[Original seen]
      ,[progresSSISdb].[dbo].[Doc].[Created on]
  FROM [progresSSISdb].[dbo].[Doc]
  join [progresSSISdb].[dbo].[Ind] on [progresSSISdb].[dbo].[Ind].[Individual ID] = [progresSSISdb].[dbo].[Doc].[Individual]
  where [Number] in (
        SELECT [Number]
        FROM [progresSSISdb].[dbo].[Doc]
        where [Document type] in (
            'Tax Registration Document',
            'National passport',
            'Birth Certificate',
            'National identity card'
        )
        AND [Created On] >= @PeriodStart
        AND [Created On] < @PeriodEndPlusOne

        INTERSECT

        SELECT [Number]
        FROM [progresSSISdb].[dbo].[Doc]
        where [Document type] in (
            'Tax Registration Document',
            'National passport',
            'Birth Certificate',
            'National identity card'
        )
        AND [Created On] < @PeriodStart    
  )
  and [Document type] in (
    'Tax Registration Document',
    'National passport',
    'Birth Certificate',
    'National identity card'
  )
  ORDER by [Number]

PRINT 'Tax ID wrong length'

SELECT [Individual]
      ,[Document category]
      ,[Document type]
      ,[Document sub type]
      ,[Number]
      ,len(Number) 'Tax ID wrong Length'
      ,[Date of issue]
      ,[Date of expiry]
      ,[Is original]
      ,[Is official]
      ,[Original seen]
      ,[Created on]
  FROM [progresSSISdb].[dbo].[Doc]
  where [Document type] = 'Tax Registration Document'
  and len(Number) <> 10
  AND [Created On] >= @PeriodStart
  AND [Created On] < @PeriodEndPlusOne

PRINT 'Passport wrong length'

SELECT [Individual]
      ,[Document category]
      ,[Document type]
      ,[Document sub type]
      ,[Number]
      ,len(Number) 'Passport wrong Length'
      ,[Date of issue]
      ,[Date of expiry]
      ,[Is original]
      ,[Is official]
      ,[Original seen]
      ,[Created on]
  FROM [progresSSISdb].[dbo].[Doc]
  where [Document type] = 'National passport'
  and len(Number) <> 8
  AND [Created On] >= @PeriodStart
  AND [Created On] < @PeriodEndPlusOne

PRINT 'Identity card wrong Length'

SELECT [Individual]
      ,[Document category]
      ,[Document type]
      ,[Document sub type]
      ,[Number]
      ,len(Number) 'Identity card wrong Length'
      ,[Date of issue]
      ,[Date of expiry]
      ,[Is original]
      ,[Is official]
      ,[Original seen]
      ,[Created on]
  FROM [progresSSISdb].[dbo].[Doc]
  where [Document type] = 'National identity card'
  and len(Number) <> 9
  AND [Created On] >= @PeriodStart
  AND [Created On] < @PeriodEndPlusOne

PRINT 'Birth Certificate wrong Length' --Exclude BCs with latin letters

SELECT [Individual]
      ,[Document category]
      ,[Document type]
      ,[Document sub type]
      ,[Number]
      ,len(Number) 'Birth Certificate wrong Length'
      ,[Date of issue]
      ,[Date of expiry]
      ,[Is original]
      ,[Is official]
      ,[Original seen]
      ,[Created on]
  FROM [progresSSISdb].[dbo].[Doc]
  where [Document type] = 'Birth Certificate'
  and len(Number) <> 10
  AND [Created On] >= @PeriodStart
  AND [Created On] < @PeriodEndPlusOne
  AND Number not like '%[a-zA-Z]%'

  PRINT 'Missing UKIDP-REG02 NO TAX ID event' --Missing event

  SELECT [Individual ID] 'Missing UKIDP-REG02 NO TAX ID event'
    FROM [progresSSISdb].[dbo].[Ind]
    where [Legacy ID] in ('na','NTID A','NTID R','NTID D','NTID L','NTID LB')
    AND [Created On] >= @PeriodStart
    AND [Created On] < @PeriodEndPlusOne

  EXCEPT

  SELECT distinct([Individual])
    FROM [progresSSISdb].[dbo].[Events]
    JOIN [progresSSISdb].[dbo].[Ind] on [progresSSISdb].[dbo].[Ind].[Individual ID] = [progresSSISdb].[dbo].[Events].[Individual]
    where [Event] = 'UKIDP-REG02'
    AND [progresSSISdb].[dbo].[Ind].[Created On] >= @PeriodStart
    AND [progresSSISdb].[dbo].[Ind].[Created On] < @PeriodEndPlusOne

PRINT 'Wrong NO TAX ID code'

  SELECT distinct([Individual]) 'Wrong NO TAX ID code' --The event without proper code
    FROM [progresSSISdb].[dbo].[Events]
    JOIN [progresSSISdb].[dbo].[Ind] on [progresSSISdb].[dbo].[Ind].[Individual ID] = [progresSSISdb].[dbo].[Events].[Individual]
    where [Event] = 'UKIDP-REG02'
    AND [progresSSISdb].[dbo].[Ind].[Created On] >= @PeriodStart
    AND [progresSSISdb].[dbo].[Ind].[Created On] < @PeriodEndPlusOne

  EXCEPT

  SELECT [Individual ID]
    FROM [progresSSISdb].[dbo].[Ind]
    where [Legacy ID] in ('na','NTID A','NTID R','NTID D','NTID L','NTID LB')
    AND [Created On] >= @PeriodStart
    AND [Created On] < @PeriodEndPlusOne

PRINT 'Result to Code correspondance check for Missing Tax IDs'

  select [Individual ID]
      ,[Full name]
      ,[Registration Group ID]
      ,[Legal Status]
      ,[Arrival date]
      ,[Legacy ID]
      ,[Result]
      ,[Result short name]
      ,'Result to Code correspondance check for Missing Tax IDs' 'Comments'
from (
    SELECT [Individual ID]
        ,[Full name]
        ,[Registration Group ID]
        ,[Legal Status]
        ,[Arrival date]
        ,[Legacy ID]
        ,[Result]
        ,[Result short name]
            ,CASE
                WHEN ([Legacy ID] = 'NTID R') and (Result = 'UKIDP-REG04') THEN 'OK'
                WHEN ([Legacy ID] = 'NTID A') and (Result = 'UKIDP-REG03') THEN 'OK'
                WHEN ([Legacy ID] = 'NTID D') and (Result = 'UKIDP-REG05') THEN 'OK'
                WHEN ([Legacy ID] = 'NTID L') and (Result = 'UKIDP-REG06') THEN 'OK'
                WHEN ([Legacy ID] = 'NTID LB') and (Result = 'UKIDP-REG07') THEN 'OK'
                ELSE 'NOK'
            END 'Result correspondance check'        
    FROM [progresSSISdb].[dbo].[Ind]
    left join [progresSSISdb].[dbo].[Events] on [progresSSISdb].[dbo].[Ind].[Individual ID] = [progresSSISdb].[dbo].[Events].Individual
              and [progresSSISdb].[dbo].[Events].[Event] = 'UKIDP-REG02'
        where [progresSSISdb].[dbo].[Ind].[Created On] >= @PeriodStart
        AND [progresSSISdb].[dbo].[Ind].[Created On] < @PeriodEndPlusOne
        AND [Legacy ID] like '%[^0-9]%'
    ) as Temp
  WHERE [Result correspondance check] = 'NOK'



Missing SPN

(3 rows affected)

IND duplicates by Tax ID

(0 rows affected)

IND duplicates by bio data

...

Individual,Document category,Document type,Document sub type,Number,Identity card wrong Length,Date of issue,Date of expiry,Is original,Is official,Original seen,Created on
FC3-110811496,Identity Documentation,National identity card,NULL,0102447169,10,2024-02-08,2028-02-08,Yes,Yes,Yes,2024-11-15 12:29:42.000
FC3-110787614,Identity Documentation,National identity card,NULL,0052214258,10,2020-09-17,2030-09-17,Yes,Yes,Yes,2024-11-14 09:32:37.000
FC3-110747000,Identity Documentation,National identity card,NULL,0062167096,10,2021-06-11,2031-06-11,Yes,Yes,Yes,2024-11-12 12:33:40.000
FC3-110727996,Identity Documentation,National identity card,NULL,0100200874,10,2023-12-08,2027-12-08,Yes,Yes,Yes,2024-11-11 13:59:42.000
FC3-110727434,Identity Documentation,National identity card,NULL,00245263,8,2018-08-02,2028-08-02,Yes,Yes,Yes,2024-11-11 13:37:12.000


Individual,Document category,Document type,Document sub type,Number,Birth Certificate wrong Length,Date of issue,Date of expiry,Is original,Is official,Original seen,Created on
FC3-110811767,Birth Documentation,Birth Certificate,NULL,А26,3,2021-03-18,NULL,Yes,Yes,Yes,2024-11-15 12:48:58.000
FC3-110811411,Birth Documentation,Birth Certificate,NULL,АЕ9543605,9,2022-07-28,NULL,Yes,Yes,Yes,2024-11-15 12:29:12.000
FC3-110809822,Birth Documentation,Birth Certificate,NULL,ІІІ-АИ800706,12,2013-07-24,NULL,Yes,Yes,Yes,2024-11-15 10:50:53.000
FC3-110792144,Birth Documentation,Birth Certificate,NULL,794/2022,8,2022-08-10,NULL,Yes,Yes,Yes,2024-11-14 14:01:48.000
FC3-110774444,Birth Documentation,Birth Certificate,NULL,І-ЖС 327343,11,2016-12-07,NULL,Yes,Yes,Yes,2024-11-13 16:47:51.000
FC3-110750401,Birth Documentation,Birth Certificate,NULL,І-ВЛ 219361,11,2010-12-15,NULL,Yes,Yes,Yes,2024-11-12 14:44:55.000
FC3-110726728,Birth Documentation,Birth Certificate,NULL,І-ЖД13854,9,2012-07-12,NULL,Yes,Yes,Yes,2024-11-11 13:09:16.000
FC3-110722407,Birth Documentation,Birth Certificate,NULL,243,3,2023-11-07,NULL,Yes,Yes,Yes,2024-11-11 09:41:49.000


Missing UKIDP-REG02 NO TAX ID event


Wrong NO TAX ID code


Individual ID,Full name,Registration Group ID,Legal Status,Arrival date,Legacy ID,Result,Result short name,Comments
FC3-110745726,"Кушпіт, Зоряна Віталіївна",FC3-24-4684978,Returned IDP,2023-11-23,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110746956,"Птиця, Валерія Олександрівна",FC3-24-4685421,Returnee,2022-09-26,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110769065,"Шмалюх, Даніела Вадимівна",FC3-24-4689470,Returned IDP,2022-05-11,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110769224,"Шмалюх, Анастасія Вадимівна",FC3-24-4689470,Returned IDP,2022-05-11,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110772125,"Давіташвілі, Грігол Валер'янович",FC3-24-4690300,Internally displaced person,2024-10-30,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110772592,"Федонкін, Анатолій Олександрович",FC3-24-4690449,IDP-like,2024-11-13,NA,UKIDP-REG09,Due to personal reasons,Result to Code correspondance check for Missing Tax IDs
FC3-110772746,"Сидоренко, Ярослава Ігорівна",FC3-24-4690533,Returnee,2022-07-25,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110792805,"Некряч, Марія Валеріївна",FC3-24-4694416,Returned IDP,2022-12-16,NTID A,NULL,NULL,Result to Code correspondance check for Missing Tax IDs
FC3-110811594,"Резников, Ярослав Романович",FC3-23-1908905,Other of concern,2024-11-15,NA,UKIDP-REG09,Due to personal reasons,Result to Code correspondance check for Missing Tax IDs
FC3-110812136,"Шевченко, Артем Максимович",FC3-24-4698085,Other of concern,2024-11-15,NA,UKIDP-REG09,Due to personal reasons,Result to Code correspondance check for Missing Tax IDs
